# Imports

In [1]:
import pandas as pd
import copy
df_raw = pd.read_csv("~/code/moth_project/0_database/tracking/track_localize_classify_annotation-2022_05_13.csv")

In [2]:
def count_species(csv):
    df = pd.read_csv(csv)
    df_count = df.groupby(['subclass']).size().sort_values(ascending=False)
    df_count.to_csv('count.txt',header=False)
    return df_count

def calculate_best_confidence_max_value(df,df_specific_id):
    max_confidence_index = df_specific_id['confidence'].idxmax()
    new_label = df.iloc[max_confidence_index,df.columns.get_loc("subclass")]
    return max_confidence_index,new_label

def modify_label_confidence(df,df_specific_id):
    max_confidence_index,new_label = calculate_best_confidence_max_value(df,df_specific_id)
    #print('Selected label ', new_label)
    for index, row in df_specific_id.iterrows():
        if index != max_confidence_index:
            df.iloc[index, df.columns.get_loc("subclass")] = new_label
            df.iloc[index, df.columns.get_loc("confidence")] = 'smoothed'
        

def smoothing_confidence_with_tracking(df_raw):
    '''
    Use the tracking insect results to smooth the classification
    
    '''
    df = df_raw.copy()
    max_id_tracked = int(df['track_id'].max())
    #max_id_tracked = 3
    for track_id in range(1,max_id_tracked+1):
        df_ids_raw = df[df['track_id'] == track_id]
        if df_ids_raw.shape[0] > 1:
            #print("\n", df_ids_raw[['subclass','confidence']])
            df_specific_id = df_ids_raw.astype({'confidence': int}) # necessary, otherwise: confidence = object instead of int
            modify_label_confidence(df,df_specific_id)
    df_final = df.mask(df == '')
    return df_final

In [3]:
df_modified = smoothing_confidence_with_tracking(df_raw)

In [5]:
""" print(df_raw.iloc[114])
print(df_modified.iloc[114]) """

' print(df_raw.iloc[114])\nprint(df_modified.iloc[114]) '

In [6]:
df_modified.to_csv('test.csv')